# Exp S2a: Deep Bimodality (λ=0.50, seeds 0-59)

## 目的
遷移中心点での大規模統計（Part 1/5）。

## 実験設計
- **λ**: 0.50（遷移中心）
- **η**: 0.4
- **Seeds**: 0-59 (60 runs)
- **S2a-e合計**: 300 seeds

## 推定時間
~60 × 11 min ≈ **11h**

## S2系列構成
- S2a: seeds 0-59 (this)
- S2b: seeds 60-119
- S2c: seeds 120-179
- S2d: seeds 180-239
- S2e: seeds 240-299

## 科学的目標
- S1cでλ=0.50が遷移領域と判明
- 300 seedsで二峰性の統計検定（Hartigan's dip test）
- ordered/collapsed比率の精密測定

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, json, time
from datetime import datetime

EXP_NAME = 'exp_S2_deep_bimodality'
NOTEBOOK_ID = 'S2a'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Notebook: {NOTEBOOK_ID} (λ=0.50, seeds 0-59)')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

In [ ]:
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.1
K = 16
NOISE_RATE = 0.4
LAMBDA_VALUE = 0.50
SEED_START = 0
SEED_END = 59

experiments = [{'lambda': LAMBDA_VALUE, 'seed': s} for s in range(SEED_START, SEED_END + 1)]
print(f'λ = {LAMBDA_VALUE}')
print(f'Seeds: {SEED_START}-{SEED_END}')
print(f'Total: {len(experiments)} runs')

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    idx = np.random.choice(len(labels), n_noisy, replace=False)
    for i in idx:
        noisy[i] = np.random.choice([l for l in range(10) if l != labels[i]])
    return noisy

def load_cifar10():
    tr = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    te = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    return torchvision.datasets.CIFAR10('./data', True, tr, download=True), torchvision.datasets.CIFAR10('./data', False, te, download=True)

def evaluate(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

In [ ]:
def train_dual_gradient(model, train_loader, test_loader, clean_labels, noisy_labels, lam):
    opt = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    sched = optim.lr_scheduler.MultiStepLR(opt, [50, 75], 0.1)
    crit = nn.CrossEntropyLoss()
    clean_t = torch.tensor(clean_labels, device=device)
    noisy_t = torch.tensor(noisy_labels, device=device)
    cached_gv = None
    step = 0
    cos_hist = []
    error_hist = []
    
    for ep in range(EPOCHS):
        model.train()
        ep_cos = []
        for x, _, idx in train_loader:
            x, idx = x.to(device), idx.to(device)
            bn, bc = noisy_t[idx], clean_t[idx]
            
            opt.zero_grad()
            loss_s = crit(model(x), bn)
            loss_s.backward(retain_graph=True)
            gs = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            if step % K == 0 or cached_gv is None:
                opt.zero_grad()
                loss_v = crit(model(x), bc)
                loss_v.backward()
                cached_gv = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            gs_n = gs / (gs.norm() + 1e-12)
            gv_n = cached_gv / (cached_gv.norm() + 1e-12)
            ep_cos.append((gs_n @ gv_n).item())
            
            g_mix = (1 - lam) * gs_n + lam * gv_n
            opt.zero_grad()
            i = 0
            for p in model.parameters():
                n = p.numel()
                p.grad = g_mix[i:i+n].view(p.shape).clone()
                i += n
            opt.step()
            step += 1
        sched.step()
        cos_hist.append(np.mean(ep_cos))
        
        if (ep + 1) % 10 == 0:
            error_hist.append({'epoch': ep + 1, 'error': 1 - evaluate(model, test_loader)})
    
    final_error = 1 - evaluate(model, test_loader)
    return final_error, np.mean(cos_hist), error_hist

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader = DataLoader(IndexedDataset(trainset), BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

m = get_resnet18().to(device)
for _ in range(20): _ = m(torch.randn(BATCH_SIZE,3,32,32,device=device))
del m; torch.cuda.empty_cache()
print('Ready')

In [ ]:
results = []
ckpt = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
done = set()

if os.path.exists(ckpt):
    results = json.load(open(ckpt))
    done = {r['seed'] for r in results}
    print(f'Loaded: {len(done)} done')

total = len(experiments)
for exp in experiments:
    lam, seed = exp['lambda'], exp['seed']
    if seed in done: continue
    
    run = len([r for r in results if SEED_START <= r['seed'] <= SEED_END]) + 1
    print(f'\n[{run}/{total}] λ={lam} seed={seed}')
    
    set_seed(seed)
    noisy = inject_label_noise(clean_labels, NOISE_RATE, seed)
    model = get_resnet18().to(device)
    
    t0 = time.time()
    err, cos, error_hist = train_dual_gradient(model, train_loader, test_loader, clean_labels, noisy, lam)
    dt = time.time() - t0
    
    phase = 'ordered' if err <= 0.20 else ('collapsed' if err >= 0.40 else 'intermediate')
    results.append({
        'experiment_id': f'{NOTEBOOK_ID}-{seed:03d}',
        'lambda': lam,
        'seed': seed,
        'final_error': err,
        'avg_cos': cos,
        'phase': phase,
        'time': dt,
        'error_history': error_hist
    })
    done.add(seed)
    
    st = '✅' if phase == 'ordered' else ('⚠️' if phase == 'collapsed' else '🔶')
    print(f'  Error: {err:.4f} | {phase} {st} | {dt/60:.1f}min')
    
    json.dump(results, open(ckpt, 'w'), indent=2)
    remaining = total - run
    print(f'  ETA: {remaining*dt/3600:.1f}h')
    
    del model; torch.cuda.empty_cache()

print('\n' + '='*50 + f'\n{NOTEBOOK_ID} DONE\n' + '='*50)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Save results for this notebook
my_results = [r for r in results if SEED_START <= r['seed'] <= SEED_END]
json.dump(my_results, open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w'), indent=2)
df = pd.DataFrame([{k: v for k, v in r.items() if k != 'error_history'} for r in my_results])
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

errors = df['final_error'].values

print('='*60)
print(f'{NOTEBOOK_ID} SUMMARY (seeds {SEED_START}-{SEED_END})')
print('='*60)
print(f'N = {len(errors)}')
print(f'Mean error: {errors.mean():.4f} ± {errors.std():.4f}')

n_ord = (df['phase'] == 'ordered').sum()
n_col = (df['phase'] == 'collapsed').sum()
n_int = (df['phase'] == 'intermediate').sum()
print(f'\nPhase distribution:')
print(f'  Ordered: {n_ord} ({100*n_ord/len(df):.1f}%)')
print(f'  Intermediate: {n_int} ({100*n_int/len(df):.1f}%)')
print(f'  Collapsed: {n_col} ({100*n_col/len(df):.1f}%)')

In [ ]:
# ===== S2系列統合分析（全ノートブック結合） =====
print('='*60)
print('S2 SERIES INTEGRATED ANALYSIS (λ=0.50)')
print('='*60)

all_files = sorted(glob.glob(f'{SAVE_DIR}/S2*_results.csv'))
if all_files:
    all_df = pd.concat([pd.read_csv(f) for f in all_files])
    all_errors = all_df['final_error'].values
    
    print(f'Total N = {len(all_errors)} (from {len(all_files)} notebooks)')
    print(f'Mean error: {all_errors.mean():.4f} ± {all_errors.std():.4f}')
    
    n_ord = (all_df['phase'] == 'ordered').sum()
    n_col = (all_df['phase'] == 'collapsed').sum()
    n_int = (all_df['phase'] == 'intermediate').sum()
    print(f'\nCombined phase distribution:')
    print(f'  Ordered: {n_ord} ({100*n_ord/len(all_df):.1f}%)')
    print(f'  Intermediate: {n_int} ({100*n_int/len(all_df):.1f}%)')
    print(f'  Collapsed: {n_col} ({100*n_col/len(all_df):.1f}%)')
    
    # Plot combined distribution
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    ax = axes[0]
    ax.hist(all_errors, bins=30, density=True, alpha=0.7, edgecolor='black')
    ax.axvline(0.20, color='green', linestyle='--', linewidth=2, label='ordered')
    ax.axvline(0.40, color='red', linestyle='--', linewidth=2, label='collapse')
    ax.set_xlabel('Final Error', fontsize=12)
    ax.set_ylabel('Density', fontsize=12)
    ax.set_title(f'S2 Combined (λ=0.50, n={len(all_errors)})', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    ax = axes[1]
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(all_errors)
    x_range = np.linspace(all_errors.min() - 0.05, all_errors.max() + 0.05, 200)
    ax.plot(x_range, kde(x_range), 'b-', linewidth=2)
    ax.fill_between(x_range, kde(x_range), alpha=0.3)
    ax.axvline(0.20, color='green', linestyle='--')
    ax.axvline(0.40, color='red', linestyle='--')
    ax.set_xlabel('Final Error', fontsize=12)
    ax.set_ylabel('Density', fontsize=12)
    ax.set_title('Kernel Density Estimate', fontsize=12)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{SAVE_DIR}/figures/S2_combined_dist.png', dpi=150)
    plt.show()
    
    # Hartigan's dip test
    try:
        from diptest import diptest
        dip_stat, p_val = diptest(all_errors)
        print(f'\nHartigan\'s Dip Test:')
        print(f'  Dip statistic: {dip_stat:.4f}')
        print(f'  p-value: {p_val:.6f}')
        print(f'  Bimodal: {"YES" if p_val < 0.05 else "NO"}')
    except ImportError:
        print('\n(diptest not installed - run: pip install diptest)')
else:
    print('No S2 results found yet')